# Scoring Data

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import pandas_profiling as pdp
from haversine import haversine

% matplotlib inline

In [9]:
%%javascript
$('.nbp-app-bar').toggle()

<IPython.core.display.Javascript object>

## Import Data

In [10]:
path = os.getcwd()

if path != '/Users/mjschillawski/Google Drive/Data/generalassembly/projects/west_nile_virus':
    path = '/Users/mjschillawski/Google Drive/Data/generalassembly/projects/west_nile_virus'
else:
    pass
path = path + '/assets/input'
os.chdir(path)

In [11]:
ls

Martinez-Prelim-EDA.ipynb
clean_test_data.csv
cleaned_data.csv*
mapdata_copyright_openstreetmap_contributors.rds
mapdata_copyright_openstreetmap_contributors.txt
noaa_weather_qclcd_documentation.pdf
sampleSubmission.csv
spray.csv
test.csv
train.csv
trap_map.png
weather.csv


In [12]:
#train = pd.read_csv('train.csv')
test_spray = pd.read_csv('spray.csv')
test_weather = pd.read_csv('weather.csv')
test = pd.read_csv('test.csv')

## Data Inspect

In [13]:
#DataInspect(test)

In [14]:
pdp.ProfileReport(test)

Number of variables,11
Number of observations,116293
Total Missing (%),0.0%
Total size in memory,9.8 MiB
Average record size in memory,88.0 B
Numeric,5
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## Fix Data

In [15]:
#train['Date'] = pd.to_datetime(train['Date'])
test_spray['Date'] = pd.to_datetime(test_spray['Date'])
test_weather['Date'] = pd.to_datetime(test_weather['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [16]:
test_spray['Date'].describe()

count                   14835
unique                     10
top       2013-08-15 00:00:00
freq                     2668
first     2011-08-29 00:00:00
last      2013-09-05 00:00:00
Name: Date, dtype: object

In [17]:
test['Date'].describe()

count                  116293
unique                     95
top       2012-07-09 00:00:00
freq                     1293
first     2008-06-11 00:00:00
last      2014-10-02 00:00:00
Name: Date, dtype: object

In [18]:
test_weather['Date'].describe()

count                    2944
unique                   1472
top       2014-08-27 00:00:00
freq                        2
first     2007-05-01 00:00:00
last      2014-10-31 00:00:00
Name: Date, dtype: object

## Drop Duplicates

In [19]:
# inspect duplicates in spray

print(test.duplicated().sum())
test[test.duplicated(keep=False)]

0


,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy


In [20]:
# inspect duplicates in spray
print(test_spray.duplicated(keep=False).sum())
test_spray[test_spray.duplicated(keep=False)][:5]

543


,Date,Time,Latitude,Longitude
484,2011-09-07,7:43:40 PM,41.983917,-87.793088
485,2011-09-07,7:43:40 PM,41.983917,-87.793088
489,2011-09-07,7:44:32 PM,41.986460,-87.794225
490,2011-09-07,7:44:32 PM,41.986460,-87.794225
491,2011-09-07,7:44:32 PM,41.986460,-87.794225


In [21]:
test_spray.drop(test_spray[test_spray.duplicated(keep='first')].index,axis=0,inplace=True)

In [22]:
print(len(test))
print(len(test_spray))
print(len(test_weather))

116293
14294
2944


## Fix weather data

In [23]:
# Changes string T to 0.005, M to 0.0 (11 observations of 2944), and everything else into a float
def rainy_day(column):
    test_weather[column] = test_weather[column].str.replace('T','0.005')
    test_weather[column] = test_weather[column].str.replace('M','0.0')
    test_weather[column] = test_weather[column].astype(float)

In [24]:
for col in ['Tavg','PrecipTotal','Depart','WetBulb','SnowFall',
            'StnPressure','SeaLevel','Depth','AvgSpeed','Heat','Cool']:
    rainy_day(col)

In [25]:
#All Tavg values that were missing (changed to 0 with rainy_day function) are given values from min/max
test_weather['Tavg'][test_weather.Tavg==0] = (test_weather['Tmin'] + test_weather['Tmax']) / 2
test_weather['Tavg'].value_counts()

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


73.0    138
70.0    117
77.0    117
75.0    110
71.0    109
74.0    107
72.0    104
78.0    103
69.0    103
76.0    100
68.0     99
79.0     98
66.0     94
67.0     89
61.0     88
64.0     86
65.0     84
80.0     84
63.0     81
62.0     67
57.0     67
60.0     61
50.0     57
81.0     55
53.0     49
54.0     49
58.0     49
55.0     48
82.0     48
56.0     46
       ... 
46.0     24
47.0     24
84.0     21
44.0     19
48.0     17
85.0     16
86.0     16
42.0     15
43.0     12
87.0      9
41.0      7
40.0      5
88.0      4
91.0      4
39.0      4
89.0      4
38.0      2
36.0      2
90.0      2
37.0      2
71.5      1
92.0      1
94.0      1
64.5      1
93.0      1
63.5      1
56.5      1
85.5      1
67.5      1
68.5      1
Name: Tavg, Length: 66, dtype: int64

## Make numeric

In [26]:
cols = [col for col in test_weather.columns if col not in ('Station','Date')]

bad_col = []

for col in cols:
    try:
        test_weather[col] = pd.to_numeric(test_weather[col])
    except:
        bad_col.append(col)
print(bad_col)

['Sunrise', 'Sunset', 'CodeSum', 'Water1']


In [27]:
test_weather.drop('CodeSum',axis=1,inplace=True)
bad_col.remove('CodeSum')

In [28]:
bad_data = {}

for col in bad_col:
    z = []
    for i in test_weather[col]:
        try:
            pd.to_numeric(i)
        except:
            if i in z:
                pass
            else:
                z.append(i)
    bad_data[col] = z
bad_data

{'Sunrise': ['-'], 'Sunset': ['-'], 'Water1': ['M']}

In [29]:
test_weather[bad_col].describe()

,Sunrise,Sunset,Water1
count,2944,2944,2944
unique,122,119,1
top,-,-,M
freq,1472,1472,2944


In [30]:
test_weather.drop(bad_col,axis=1,inplace=True)

## Date Engineering

In [31]:
# need to do something that incorporates date information
# probably categorical?
# other options? seasonal, weekly
# continuous but circular

# continuous, as day of year
test["dayofyear"] = test['Date'].dt.dayofyear

# month
# train['month'] = train['Date'].dt.month

# quarter
# train['quarter'] = train['Date'].dt.quarter

## Dummy species

In [32]:
# get dummies for mosquito species

test = pd.get_dummies(test, columns=['Species'])

## Join weather data to trap data

In [33]:
print(len(test),len(test_weather))

116293 2944


In [34]:
test_weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Depth', 'SnowFall', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [35]:
station1 = test_weather[test_weather['Station']==1].copy()
station2 = test_weather[test_weather['Station']==2].copy()

#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
station1['Latitude'] = 41.995
station1['Longitude'] = -87.9336

#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level
station2['Latitude'] = 41.78611
station2['Longitude'] = -87.75222

In [37]:
stations = pd.merge(station1,station2,on='Date',suffixes=('_s1','_s2'))
testtrapsweather = pd.merge(test,stations,on='Date')

#DataInspect(testtrapsweather)

## Calculate point estimates of weather data at trap location

In [38]:
# calculate distance of traps to weather stations
dist_1 = np.sqrt(((testtrapsweather['Latitude'] - testtrapsweather['Latitude_s1'])**2 + 
 (testtrapsweather['Longitude'] - testtrapsweather['Longitude_s1'])**2))

dist_2 = np.sqrt(((testtrapsweather['Latitude'] - testtrapsweather['Latitude_s2'])**2 + 
 (testtrapsweather['Longitude'] - testtrapsweather['Longitude_s2'])**2))

# calculate distance weights to each trap
# to weight the weather data by proximity
total_dist = dist_1 + dist_2
testtrapsweather['weight_1'] = dist_1 / total_dist
testtrapsweather['weight_2'] = dist_2 / total_dist

In [39]:
# Apply distance weights to weather data
# Inverse weight because the closer station should have the heavier weight

station1_list = [col for col in testtrapsweather.columns 
                if '_s1' in col and col not in ('Station_s1','Latitude_s1','Longitude_s1')]
station2_list = [col for col in testtrapsweather.columns 
                 if '_s2' in col and col not in ('Station_s2','Latitude_s2','Longitude_s2')]

for col in station1_list:
    testtrapsweather[col] = testtrapsweather['weight_2'] * testtrapsweather[col]
for col in station2_list:
    testtrapsweather[col] = testtrapsweather['weight_1'] * testtrapsweather[col]

In [40]:
testtrapsweather.columns

Index(['Id', 'Date', 'Address', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'dayofyear', 'Species_CULEX ERRATICUS', 'Species_CULEX PIPIENS',
       'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS',
       'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS',
       'Species_CULEX TERRITANS', 'Species_UNSPECIFIED CULEX', 'Station_s1',
       'Tmax_s1', 'Tmin_s1', 'Tavg_s1', 'Depart_s1', 'DewPoint_s1',
       'WetBulb_s1', 'Heat_s1', 'Cool_s1', 'Depth_s1', 'SnowFall_s1',
       'PrecipTotal_s1', 'StnPressure_s1', 'SeaLevel_s1', 'ResultSpeed_s1',
       'ResultDir_s1', 'AvgSpeed_s1', 'Latitude_s1', 'Longitude_s1',
       'Station_s2', 'Tmax_s2', 'Tmin_s2', 'Tavg_s2', 'Depart_s2',
       'DewPoint_s2', 'WetBulb_s2', 'Heat_s2', 'Cool_s2', 'Depth_s2',
       'SnowFall_s2', 'PrecipTotal_s2', 'StnPressure_s2', 'SeaLevel_s2',
       'ResultSpeed_s2', 'ResultDir_s2', 'AvgSpeed_s2', 'Latitude_s2',
       'Longitude_s2', 

In [41]:
# we are adding the weighted station1 and station2 weather data
# and dropping the partial columns

for col in [col for col in testtrapsweather.columns 
            if 's1' in col and col not in ('Station_s1','Latitude_s1','Longitude_s1')]:
    name = col.replace('_s1','')
    testtrapsweather[name] = testtrapsweather[col] + testtrapsweather[name+'_s2']
    testtrapsweather.drop([col,name+'_s2'],axis=1,inplace=True)

In [42]:
testtrapsweather.columns

Index(['Id', 'Date', 'Address', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'dayofyear', 'Species_CULEX ERRATICUS', 'Species_CULEX PIPIENS',
       'Species_CULEX PIPIENS/RESTUANS', 'Species_CULEX RESTUANS',
       'Species_CULEX SALINARIUS', 'Species_CULEX TARSALIS',
       'Species_CULEX TERRITANS', 'Species_UNSPECIFIED CULEX', 'Station_s1',
       'Latitude_s1', 'Longitude_s1', 'Station_s2', 'Latitude_s2',
       'Longitude_s2', 'weight_1', 'weight_2', 'Tmax', 'Tmin', 'Tavg',
       'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Depth', 'SnowFall',
       'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir',
       'AvgSpeed'],
      dtype='object')

In [43]:
# Drop station information columns
col1 = [col for col in testtrapsweather.columns if '_s1' in col]
col2 = [col for col in testtrapsweather.columns if '_s2' in col]
cols = col1 + col2

testtrapsweather.drop(cols,axis=1,inplace=True)

## Spraying data

### Feature Engineering: Binary Flag - Whether Trap affected by Spraying w/in Last 6 months/0.5 mile

In [97]:
# we calculate the haversine (great circle) distance between the spray intervention and trap site
# we calculate the time (days) between the spray intervention and trap site
# together, we create a binary flag for denoting whether the intervention occurred within a recent window
# relative to the trap observation, both in time and space
# e.g. within 1 week and within 0.25 miles
# these we will iterate through our model, and measure the effectiveness of intervention, through the 
# increase/decrease of WNV probability at our trap sites.

from haversine import haversine

print('starting...')

inputs = testtrapsweather.index

def distance_calc(i): 
    distance_binary = []
    
    temp_lat = testtrapsweather.at[i,'Latitude']
    temp_long = testtrapsweather.at[i,'Longitude']

    # calculate distance from traps to spray locations
    dists = []
    if i % 500 == 0:
        print(i)
    for s in test_spray.index:
        dist = haversine((test_spray.at[s,'Latitude'],test_spray.at[s,'Longitude']),(temp_lat,temp_long),miles=True)
        dists.append(dist)
    
    return dists

def time_calc(i): 
    time_binary = []
        
    # calculate time since spray
    time_since_spray = testtrapsweather.at[i,'Date'] - test_spray['Date']
    time_since_spray = time_since_spray.dt.total_seconds()
    time_since_spray = (((time_since_spray/60)/60)/24)
    
    return time_since_spray

distance_binary = [distance_calc(i) for i in inputs[:5]]
time_binary = [time_calc(i) for i in inputs[:5]]

distance_binary = pd.DataFrame(distance_binary)
time_binary = pd.DataFrame(time_binary)

time_binary.reset_index(inplace=True)
time_binary.drop('index',axis=1,inplace=True)

# if observation took place before spray, zero out time
# else return elapsed time between spray and observation

for col in time_binary.columns:
    time_binary[col] = time_binary[col].map(lambda x: 0 if x < 0 else x)
    
# https://chrisalbon.com/python/data_wrangling/pandas_rename_multiple_columns/
time_binary.columns = distance_binary.columns

time_binary_backup = time_binary.copy()
distance_binary_backup = distance_binary.copy()

starting...
0


In [98]:
time_tp = time_binary.transpose()
distance_tp = distance_binary.transpose()

binary = pd.merge(distance_tp,time_tp,how='inner',left_index=True,right_index=True,suffixes=('_d','_t'))
binary.shape

(14294, 10)

In [102]:
#THIS IS A COMPUTATIONALLY INTENSIVE CELL
# create a bunch of dummy variables to evaluate decaying spray effect

values = []


inputs = testtrapsweather.index

def CalculateBinary(i):
    observations = []
    if i % 500 == 0:
        print(i)

    d = str(i) + '_d'
    t = str(i) + '_t'
    
    cols = binary[[d,t]]
    
    if len(cols[np.logical_and(np.logical_and(cols[d] <= 0.5,cols[d] > 0),
       np.logical_and(cols[t] <= 7,cols[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
    
    return observations

values = [CalculateBinary(i) for i in inputs[:5]]
values

0


[[0], [0], [0], [0], [0]]

In [92]:
binary[np.logical_and(np.logical_and(binary['0_d'] <= 0.5,binary['0_d'] > 0),
       np.logical_and(binary['0_t'] <= 7,binary['0_t'] > 0))]

,0_d,1_d,0_t,1_t


In [93]:
if len(binary[np.logical_and(np.logical_and(binary[d] <= 1,binary[d] > 0),
           np.logical_and(binary[t] <= 7,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
    if len(binary[np.logical_and(np.logical_and(binary[d] <= 5,binary[d] > 0),
       np.logical_and(binary[t] <= 7,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
        
    if len(binary[np.logical_and(np.logical_and(binary[d] <= 0.5,binary[d] > 0),
           np.logical_and(binary[t] <= 30,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
    if len(binary[np.logical_and(np.logical_and(binary[d] <= 1,binary[d] > 0),
           np.logical_and(binary[t] <= 30,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
    if len(binary[np.logical_and(np.logical_and(binary[d] <= 5,binary[d] > 0),
           np.logical_and(binary[t] <= 30,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)

    if len(binary[np.logical_and(np.logical_and(binary[d] <= 0.5,binary[d] > 0),
           np.logical_and(binary[t] <= 90,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
    if len(binary[np.logical_and(np.logical_and(binary[d] <= 1,binary[d] > 0),
           np.logical_and(binary[t] <= 90,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)
    if len(binary[np.logical_and(np.logical_and(binary[d] <= 5,binary[d] > 0),
           np.logical_and(binary[t] <= 90,binary[t] > 0))]) > 0:
        observations.append(1)
    else:
        observations.append(0)

[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [ ]:
values = pd.DataFrame(values)

spray_col = ['1week_halfmile','1week_1mile','1week_5mile',
            '1month_halfmile','1month_1mile','1month_5mile',
            '1quarter_halfmile','1quarter_1mile','1quarter_5mile']

values.columns = spray_col

values.to_csv('spray_data_binary.csv')

In [ ]:
data = pd.merge(testtrapsweather,values,left_index=True,right_index=True)

## Export cleaned test data

In [ ]:
data.to_csv('cleaned_data.csv')

## Scoring

## Export scored data

In [ ]:
ids = clean_test_data['Id']

predictions = pd.DataFrame(gb_probas[:,1],index=ids)
predictions.rename(columns={0:'WnvPresent'},inplace=True)

In [ ]:
path = '/Users/mjschillawski/Google Drive/Data/generalassembly/projects/west_nile_virus/assets/output'
predictions.to_csv(path+'predictions_'+model_name+'.csv')